In [19]:
import yfinance as yf 
from openai import OpenAI
import os 
import json 
import sys 
import requests
sys.path.append(r"C:\Users\o\agentflightbooking\fiancialAIAgent")

In [21]:
from openai import OpenAI
deepseek_client = OpenAI(
    api_key=os.getenv("DEEPSEEK_API_KEY"),
    base_url=os.getenv("DEEPSEEK_BASE_URL")
)

In [47]:
def search_ticker_finnhub(company_name, api_key):
    url = f"https://finnhub.io/api/v1/search?q={company_name}&token={api_key}"
    response = requests.get(url)
    return response.json()['result'][0]['displaySymbol']


In [48]:
response = search_ticker_finnhub("Allianz SE",os.getenv('FINHUB_API_key'))

In [49]:
response

'ALV.DE'

In [50]:
ticker = search_ticker_finnhub("Allianz SE", os.getenv('FINHUB_API_key'))

In [51]:
ticker

'ALV.DE'

In [65]:
from datetime import datetime
import pandas as pd


def get_stock_price(company_name, start_date, end_date):
    api_key = os.getenv('FINHUB_API_key')
    if not api_key:
        raise ValueError("FINHUB_API_key environment variable is not set.")
    
    ticker = search_ticker_finnhub(company_name, api_key)
    
    start_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_dt = datetime.strptime(end_date, '%Y-%m-%d')
    
    stock_data = yf.download(ticker, start=start_dt, end=end_dt)
    if stock_data.empty:
        raise ValueError(f"No stock data found for ticker {ticker} between {start_date} and {end_date}")
    
    return stock_data['Close']

In [67]:
get_stock_price("Apple", "2023-01-01", "2023-10-01")

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


ValueError: No stock data found for ticker AAPL between 2023-01-01 and 2023-10-01

In [70]:
import requests

url = "https://query1.finance.yahoo.com/v8/finance/chart/ALV.DE"
params = {
    "period1": 1672531200,  # timestamp Unix début (ex: 2023-01-01)
    "period2": 1698796800,  # timestamp Unix fin (ex: 2023-10-01)
    "interval": "1d"
}

response = requests.get(url, params=params)
data = response.json()
print(data)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [106]:
import os
import requests

def get_financial_reports(company_name, api_key=os.getenv('FINHUB_API_key')):
    ticker = search_ticker_finnhub(company_name, api_key)
    url = f"https://finnhub.io/api/v1/stock/financials?symbol={ticker}&token={api_key}"
    response = requests.get(url)
    data = response.json()
    return data


In [103]:
get_financial_reports = get_financial_reports(company_name="NVIDIA", api_key=os.getenv('FINHUB_API_key'))

In [104]:
get_financial_reports

{'error': "You don't have access to this resource."}

In [ ]:
import os
import requests

def get_ticker(company_name, api_key=None):
    if api_key is None:
        api_key = os.getenv("FINHUB_API_key")
    url = f"https://finnhub.io/api/v1/search?q={company_name}&token={api_key}"
    response = requests.get(url)
    data = response.json()
    if "result" in data and len(data["result"]) > 0:
        return data["result"][0]["symbol"]
    raise ValueError(f"No ticker found for company {company_name}")

def get_financial_reports(ticker, api_key=None):
    if api_key is None:
        api_key = os.getenv("FINHUB_API_key")
    url = f"https://finnhub.io/api/v1/stock/financials-reported?symbol={ticker}&token={api_key}"
    response = requests.get(url)
    data = response.json()
    if "data" not in data or not data["data"]:
        raise ValueError("No financial reports found.")
    return data["data"]

def get_company_cik(ticker, api_key=None):
    if api_key is None:
        api_key = os.getenv("FINHUB_API_key")
    url = f"https://finnhub.io/api/v1/stock/profile2?symbol={ticker}&token={api_key}"
    response = requests.get(url)
    data = response.json()
    return data.get("cik", None)

# Exemple d’utilisation
if __name__ == "__main__":
    api_key = os.getenv("FINHUB_API_key")
    company_name = "Apple Inc"
    ticker = get_ticker(company_name, api_key)
    print(f"Ticker: {ticker}")

    cik = get_company_cik(ticker, api_key)
    print(f"CIK: {cik}")

    financial_reports = get_financial_reports(ticker, api_key)
    print(f"Nombre de rapports financiers trouvés : {len(financial_reports)}")
    for report in financial_reports[:3]:  # affiche les 3 premiers rapports
        print(f"Date rapport: {report['reportDate']}, Type: {report['fiscalPeriod']}")


In [112]:
import os
import requests

def get_ticker_and_cik(company_name, api_key=None):
    if api_key is None:
        api_key = os.getenv("FINHUB_API_key")
    search_url = f"https://finnhub.io/api/v1/search?q={company_name}&token={api_key}"
    response = requests.get(search_url)
    data = response.json()
    if not data['result']:
        raise ValueError("Aucun ticker trouvé")
    ticker = data['result'][0]['symbol']

    profile_url = f"https://finnhub.io/api/v1/stock/profile?symbol={ticker}&token={api_key}"
    response = requests.get(profile_url)
    profile = response.json()
    cik = profile.get("cik", None)
    
    return ticker, cik

def get_financials_from_cik_or_ticker(ticker=None, cik=None, freq="quarterly", api_key=None):
    if api_key is None:
        api_key = os.getenv("FINHUB_API_key")

    if cik:
        url = f"https://finnhub.io/api/v1/stock/financials-reported?cik={cik}&freq={freq}&token={api_key}"
    elif ticker:
        url = f"https://finnhub.io/api/v1/stock/financials-reported?symbol={ticker}&freq={freq}&token={api_key}"
    else:
        raise ValueError("Ticker ou CIK requis")
    
    response = requests.get(url)
    data = response.json()

    if "data" not in data or not data["data"]:
        raise ValueError("Aucun rapport financier trouvé")

    return data["data"]


In [114]:
get_ticker_and_cik("APPLE", os.getenv('FINHUB_API_key'))

('AAPL', None)

In [ ]:
def search_edgar_filings(ticker: str, form_type: str = "10-K"):
    """
    Recherche les dépôts EDGAR pour un ticker donné
    
    Args:
        ticker: Symbole boursier
        form_type: Type de formulaire (10-K, 10-Q, 8-K, etc.)
    
    Returns:
        Liste des dépôts trouvés
    """
    url = "https://efts.sec.gov/LATEST/search/index"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
        'Accept': 'application/json'
    }
    
    params = {
        'q': ticker,
        'dateRange': 'last5years',
        'category': 'form',
        'forms': form_type,
        'startdt': '2020-01-01',
        'enddt': '2024-12-31'
    }
    
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        
        filings = data.get('hits', {}).get('hits', [])
        print(f"✓ Trouvé {len(filings)} dépôts EDGAR pour {ticker}")
        return filings
    except requests.RequestException as e:
        print(f"✗ Erreur recherche EDGAR pour {ticker}: {e}")
        return []

In [116]:
fin_cik_edgar("Apple Inc", os.getenv('FINHUB_API_key'))

ValueError: Erreur lors de la récupération des données : 403